# Visualize Taxonomy

We use the [*networkx*](https://networkx.github.io/) Python package for analyzing graphs and the [*graphviz*](https://graphviz.readthedocs.io/) Python package for drawing graphs.

See http://www.graphviz.org/doc/info/attrs.html for a description of the different graphviz attributes.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from typing import Any, Mapping, Tuple

import graphviz as gv
import networkx as nx
import pandas as pd

from taxonomy_mapping.taxonomy_graph import TaxonNode, build_taxonomy_graph

In [ ]:
TAXONOMY_CSV_PATH = r'g:/temp/lila/lila-taxonomy-mapping_release.csv'
OUTPUT_FOLDER = r'g:\temp'

## Load taxonomy CSV

In [ ]:
df = pd.read_csv(TAXONOMY_CSV_PATH)
display(df.head())

## Show rows without a taxonomy string

In [ ]:
# This should just be things like "end" and "fire"

with pd.option_context('display.max_rows', 100):
     display(df[df['taxonomy_string'].isna()])

## Build the taxonomy graph

In [ ]:
graph, taxon_to_node, label_to_node = build_taxonomy_graph(df)

## Graph helper methods

In [ ]:
def nx_to_gv(nx_g: nx.DiGraph, vert_spacing: float = 0.5) -> gv.Digraph:
    """Converts a networkX graph to a graphviz graph."""
    gv_g = gv.Digraph(
        graph_attr=dict(overlap='false', concentrate='true', ranksep=str(vert_spacing)),
        node_attr=dict(margin='0', width='0', height='0'))
    for n in nx_g.nodes:  # each node is a TaxonNode
        gv_g.node(f'{n.level}\n{n.name}')
    for n1, n2 in nx_g.edges:
        n1_id = f'{n1.level}\n{n1.name}'
        n2_id = f'{n2.level}\n{n2.name}'
        gv_g.edge(n1_id, n2_id)
    return gv_g


def visualize_subgraph(nx_graph: nx.DiGraph, node: Any,
                       vert_spacing: float = 0.5) -> gv.Digraph:
    """Returns a gv.Digraph rooted at node."""
    nx_subgraph_nodes = nx.descendants(nx_graph, node) | {node}
    nx_subgraph = nx_graph.subgraph(nx_subgraph_nodes)
    gv_subgraph = nx_to_gv(nx_subgraph, vert_spacing=vert_spacing)
    return gv_subgraph

## Render entire Taxonomy graph to SVG

May include disconnected components.

In [ ]:
import os
gv_g = nx_to_gv(graph, vert_spacing=1.5)
gv_g.render(os.path.join(OUTPUT_FOLDER,'taxonomy-rendering'), format='svg', cleanup=True)
gv_g.render(os.path.join(OUTPUT_FOLDER,'taxonomy-rendering'), format='jpg', cleanup=True)
gv_g.render(os.path.join(OUTPUT_FOLDER,'taxonomy-rendering'), format='pdf', cleanup=True)
# display(gv_g)  # this can be huge

## Analyze Taxonomy graph

In [ ]:
print('Total number of nodes:', len(graph.nodes))
print('Number of disconnected components:', nx.number_weakly_connected_components(graph))

for subgraph in nx.weakly_connected_components(graph):
    # each subgraph is a set of (taxon_level, taxon_name)

    # get root node
    subgraph = graph.subgraph(subgraph)
    root_nodes = [n for n, d in subgraph.in_degree() if d==0]
    assert len(root_nodes) == 1
    root_node = root_nodes[0]

    print(f'Graph rooted at {root_node} has {len(subgraph.nodes)} nodes')

### Display disconnected component

In [ ]:
gv_g = visualize_subgraph(graph, node=taxon_to_node[('phylum', 'tracheophyta')],
                          vert_spacing=0.2)
display(gv_g)

## Query the graph

### Show the smallest subgraph containing all given dataset labels

In [ ]:
labels = [
    ('Idaho Camera Traps', 'mountain_lion'),
    ('Idaho Camera Traps', 'bobcat'),
    ('Idaho Camera Traps', 'cat_domestic'),
    ('Idaho Camera Traps', 'lynx'),
    ('Idaho Camera Traps', 'lion')
]
nodes = [label_to_node[label] for label in labels]
lca_node = TaxonNode.lowest_common_ancestor(set(nodes))
assert lca_node is not None
print(lca_node)
gv_g = visualize_subgraph(graph, node=taxon_to_node[(lca_node.level, lca_node.name)],
                          vert_spacing=0.5)
display(gv_g)

### Get the set of dataset labels corresponding to this subgraph

In [ ]:
lca_node.get_dataset_labels()